# Setting up libraries

In [2]:
!pip install ortools

from ortools.sat.python import cp_model
import csv

# Define the initial setup
num_shifts = 3
num_days = 56  # Updated to 56 days


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires keras<2.14,>=2.13.1, but you have keras 2.14.0 which is incompatible.
tensorflow-intel 2.13.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.27.3 which is incompatible.
C:\Users\91877\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\91877\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 130.2/130.2 MB 5.5 MB/s eta 0:00:00
     -------------------------------------- 133.7/133.7 kB 7.7 MB/s eta 0:00:00
     ------------------------------------- 426.9/426.9 kB 13.0 MB/s eta 0:00:00
     --------------------------------------- 11.6/11.6 MB 11.1 MB/s eta 0:00:00
     ------------------------------------- 345.4/345.4 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.24.0
    Uninstalling protobuf-4.24.0:
      Successfully uninstalled protobuf-4.24.0
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3


# Importing Nurse data from Nurse.csv

In [5]:
# Load nurses data from CSV
nurses = []
teams = []
with open('Nurses.csv', mode='r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        nurses.append(row['Nurse'])
        teams.append(row['Team'])

num_nurses = len(nurses)
all_nurses = range(num_nurses)
all_days = range(num_days)
all_shifts = range(num_shifts)


In [6]:
# Creates the model.
model = cp_model.CpModel()

# Creates shift variables.
shifts = {}
for n in all_nurses:
    for d in all_days:
        for s in all_shifts:
            shifts[(n, d, s)] = model.NewBoolVar(f"shift_n{n}_d{d}_s{s}")


# Adding Constraints

Each Shift is Assigned to Exactly One Nurse per Day

In [7]:
# Each shift is assigned to exactly one nurse in the schedule period.
for d in all_days:
    for s in all_shifts:
        model.AddExactlyOne(shifts[(n, d, s)] for n in all_nurses)


Each Nurse Works at Most One Shift per Day

In [8]:
# Each nurse works at most one shift per day.
for n in all_nurses:
    for d in all_days:
        model.AddAtMostOne(shifts[(n, d, s)] for s in all_shifts)


Distribute Shifts Evenly Among Nurses

In [10]:
min_shifts_per_nurse = (num_shifts * num_days) // num_nurses
max_shifts_per_nurse = min_shifts_per_nurse + (num_shifts * num_days) % num_nurses

for n in all_nurses:
    shifts_worked = []
    for d in all_days:
        for s in all_shifts:
            shifts_worked.append(shifts[(n, d, s)])
    model.Add(min_shifts_per_nurse <= sum(shifts_worked))
    model.Add(sum(shifts_worked) <= max_shifts_per_nurse)



Ensuring Fair Distribution of Weekend Shifts

In [11]:
weekend_days = [d for d in all_days if d % 7 in [5, 6]]  # Saturday = 5, Sunday = 6
min_weekend_shifts = (len(weekend_days) * num_shifts) // num_nurses
max_weekend_shifts = min_weekend_shifts + ((len(weekend_days) * num_shifts) % num_nurses)

for n in all_nurses:
    weekend_shifts_worked = []
    for d in weekend_days:
        for s in all_shifts:
            weekend_shifts_worked.append(shifts[(n, d, s)])
    model.Add(min_weekend_shifts <= sum(weekend_shifts_worked))
    model.Add(sum(weekend_shifts_worked) <= max_weekend_shifts)


Avoid Consecutive Shift 0 and 1 Assignments

In [12]:
for n in all_nurses:
    for d in range(num_days - 1):
        model.AddBoolOr([shifts[(n, d, 0)].Not(), shifts[(n, d + 1, 0)].Not()])
        model.AddBoolOr([shifts[(n, d, 1)].Not(), shifts[(n, d + 1, 1)].Not()])


Ensure Shift 2 Continuity

In [13]:
for n in all_nurses:
    for d in range(0, num_days, 7):  # Iterate over each Monday
        if d + 2 < num_days:  # Ensure we don't go out of bounds
            model.Add(shifts[(n, d, 2)] == shifts[(n, d + 1, 2)])
            model.Add(shifts[(n, d + 1, 2)] == shifts[(n, d + 2, 2)])


Limit Each Nurse to 4 Shifts Per Week

In [14]:
for n in all_nurses:
    for w in range(0, num_days, 7):  # Each week
        week_shifts = []
        for d in range(w, min(w + 7, num_days)):
            for s in all_shifts:
                week_shifts.append(shifts[(n, d, s)])
        model.Add(sum(week_shifts) <= 4)


Ensure Team A Nurses Do Not Work Shift 0

In [15]:
team_A_nurses = [i for i, team in enumerate(teams) if team == 'A']

for n in team_A_nurses:
    for d in all_days:
        model.Add(shifts[(n, d, 0)] == 0)


Ensure No More Than One Nurse per Team Works Each Day

In [16]:
for d in all_days:
    for team in set(teams):
        team_nurses = [i for i, t in enumerate(teams) if t == team]
        for s in all_shifts:
            model.Add(sum(shifts[(n, d, s)] for n in team_nurses) <= 1)


# Defining Solution to printer

In [17]:
class NursesPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, shifts, num_nurses, num_days, num_shifts, limit, nurses):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._num_nurses = num_nurses
        self._num_days = num_days
        self._num_shifts = num_shifts
        self._solution_count = 0
        self._solution_limit = limit
        self._nurses = nurses

    def on_solution_callback(self):
        self._solution_count += 1
        print(f"Solution {self._solution_count}")
        for d in range(self._num_days):
            print(f"Day {d + 1}")
            for n in range(self._num_nurses):
                is_working = False
                for s in range(self._num_shifts):
                    if self.Value(self._shifts[(n, d, s)]):
                        is_working = True
                        print(f"  Nurse {self._nurses[n]} works shift {s}")
                if not is_working:
                    print(f"  Nurse {self._nurses[n]} does not work")
        if self._solution_count >= self._solution_limit:
            print(f"Stop search after {self._solution_limit} solutions")
            self.StopSearch()

    def solution_count(self):
        return self._solution_count


# Solving the model and displaying results 

In [18]:
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
solver.parameters.enumerate_all_solutions = True

solution_limit = 5
solution_printer = NursesPartialSolutionPrinter(shifts, num_nurses, num_days, num_shifts, solution_limit, nurses)
solver.Solve(model, solution_printer)

# Print statistics
print(f"Statistics")
print(f"  - conflicts      : {solver.NumConflicts()}")
print(f"  - branches       : {solver.NumBranches()}")
print(f"  - wall time      : {solver.WallTime()} s")
print(f"  - solutions found: {solution_printer.solution_count()}")


Solution 1
Day 1
  Nurse 0 does not work
  Nurse 1 works shift 1
  Nurse 2 does not work
  Nurse 3 does not work
  Nurse 4 does not work
  Nurse 5 works shift 0
  Nurse 6 works shift 2
  Nurse 7 does not work
  Nurse 8 does not work
  Nurse 9 does not work
Day 2
  Nurse 0 works shift 1
  Nurse 1 does not work
  Nurse 2 does not work
  Nurse 3 works shift 0
  Nurse 4 does not work
  Nurse 5 does not work
  Nurse 6 works shift 2
  Nurse 7 does not work
  Nurse 8 does not work
  Nurse 9 does not work
Day 3
  Nurse 0 does not work
  Nurse 1 works shift 1
  Nurse 2 does not work
  Nurse 3 does not work
  Nurse 4 works shift 0
  Nurse 5 does not work
  Nurse 6 works shift 2
  Nurse 7 does not work
  Nurse 8 does not work
  Nurse 9 does not work
Day 4
  Nurse 0 works shift 1
  Nurse 1 does not work
  Nurse 2 does not work
  Nurse 3 works shift 0
  Nurse 4 does not work
  Nurse 5 does not work
  Nurse 6 does not work
  Nurse 7 does not work
  Nurse 8 works shift 2
  Nurse 9 does not work
Day 5